# UCI - Human Action Recognition - Deep-Learning

### import

In [34]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import tensorflow as tf

In [6]:
np.random.seed(42)
tf.set_random_seed(42)

In [39]:
# Importing libraries
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Conv1D, MaxPooling1D, Flatten
from keras.layers.core import Dense, Dropout

### Load Data

In [10]:
# Data directory
DATADIR = 'UCI HAR Dataset'

In [11]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [20]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'{DATADIR}/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [21]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'{DATADIR}/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [14]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [15]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [17]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [18]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [22]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [23]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [41]:
X_train.shape

(7352, 128, 9)

## Defining LSTM Network model:

In [25]:
# Initiliazing the sequential model
model1 = Sequential()
# Configuring the parameters
model1.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model1.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model1.add(Dense(n_classes, activation='sigmoid'))
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compiling the model
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [27]:
# Training the model
model1.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 15s 2ms/step - loss: 1.3008 - accuracy: 0.4645 - val_loss: 1.0891 - val_accuracy: 0.5565
Epoch 2/30
7352/7352 [==============================] - 14s 2ms/step - loss: 0.9056 - accuracy: 0.6171 - val_loss: 0.8277 - val_accuracy: 0.5942
Epoch 3/30
7352/7352 [==============================] - 15s 2ms/step - loss: 0.7434 - accuracy: 0.6549 - val_loss: 0.7569 - val_accuracy: 0.6230
Epoch 4/30
7352/7352 [==============================] - 14s 2ms/step - loss: 0.6725 - accuracy: 0.6800 - val_loss: 0.6941 - val_accuracy: 0.6651
Epoch 5/30
7352/7352 [==============================] - 14s 2ms/step - loss: 0.6236 - accuracy: 0.7116 - val_loss: 0.6568 - val_accuracy: 0.7326
Epoch 6/30
7352/7352 [==============================] - 15s 2ms/step - loss: 0.5866 - accuracy: 0.7333 - val_loss: 0.7696 - val_accu

In [38]:
# Testing
score = model1.evaluate(X_test, Y_test)
print("Accuracy: ", score[1])
print("Loss: ", score[0])
print(confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(model1.predict(X_test), axis=1)))

2947/2947 [==============================] - 1s 258us/step
Accuracy:  0.8954869508743286
Loss:  0.41655886096877154
[[454  27  15   0   0   0]
 [  4 429  31   7   0   0]
 [  0   1 419   0   0   0]
 [  1   1   1 381 105   2]
 [  0   0   0  86 446   0]
 [  0   0   0   0  27 510]]


## Defining CNN-LSTM Network model:

In [42]:
n_steps, n_length = 4, 32
X_train = X_train.reshape((X_train.shape[0], n_steps, n_length, input_dim))
X_test = X_test.reshape((X_test.shape[0], n_steps, n_length, input_dim))

In [43]:
# define model
# CNN layers for feature extraction on input data combined with LSTMs to support sequence prediction
model2 = Sequential()
# CNN layers
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,input_dim)))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model2.add(TimeDistributed(Dropout(0.5)))
model2.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model2.add(TimeDistributed(Flatten()))
# LSTM layer
model2.add(LSTM(n_hidden))
model2.add(Dropout(0.5))
model2.add(Dense(n_hidden, activation='relu'))
model2.add(Dense(n_classes, activation='sigmoid'))
model2.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, None, 30, 64)      1792      
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 28, 64)      12352     
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 28, 64)      0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 14, 64)      0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 896)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                118912    
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)              

In [45]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
# Training the model
model2.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 7s 960us/step - loss: 0.7108 - accuracy: 0.7225 - val_loss: 0.5752 - val_accuracy: 0.7784
Epoch 2/30
7352/7352 [==============================] - 7s 944us/step - loss: 0.2978 - accuracy: 0.8980 - val_loss: 0.3713 - val_accuracy: 0.8724
Epoch 3/30
7352/7352 [==============================] - 7s 993us/step - loss: 0.1675 - accuracy: 0.9400 - val_loss: 0.3436 - val_accuracy: 0.8951
Epoch 4/30
7352/7352 [==============================] - 7s 908us/step - loss: 0.1412 - accuracy: 0.9479 - val_loss: 0.3644 - val_accuracy: 0.8880
Epoch 5/30
7352/7352 [==============================] - 7s 895us/step - loss: 0.1392 - accuracy: 0.9453 - val_loss: 0.3671 - val_accuracy: 0.8904
Epoch 6/30
7352/7352 [==============================] - 7s 895us/step - loss: 0.1290 - accuracy: 0.9486 - val_loss: 0.2950 - val_accuracy: 0.9152
Epoch 7/30
7352/7352 [==============================] - 7s 898us/step - loss

In [50]:
# Testing
score = model2.evaluate(X_test, Y_test)
print("Accuracy: ", score[1])
print("Loss: ", score[0])
print(confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(model2.predict(X_test), axis=1)))

2947/2947 [==============================] - 0s 158us/step
Accuracy:  0.8988802433013916
Loss:  0.6556248250248767
[[465   0  31   0   0   0]
 [ 10 428  33   0   0   0]
 [  2   0 418   0   0   0]
 [  0   3   0 428  60   0]
 [  1   0   0 132 399   0]
 [  0  26   0   0   0 511]]
